In [1]:
import requests
import lxml
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
#再来请求一次
html = requests.get('https://movie.douban.com/top250',headers = headers)
html.text[:200]

'<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <m'

In [2]:
from lxml import etree

bs = etree.HTML(html.text) 
#bs.xpath('//tr[@class = "item"]/td[2]/p')[0].text

In [22]:
bs.xpath('//div [@class = "item"]')

[<Element div at 0x1712b175688>,
 <Element div at 0x1712c7519c8>,
 <Element div at 0x1712c652b88>,
 <Element div at 0x1712c735608>,
 <Element div at 0x1712c7358c8>,
 <Element div at 0x1712c74aec8>,
 <Element div at 0x1712c74acc8>,
 <Element div at 0x1712c721788>,
 <Element div at 0x1712c721a08>,
 <Element div at 0x1712c7357c8>,
 <Element div at 0x1712c721588>,
 <Element div at 0x1712c721648>,
 <Element div at 0x1712c721248>,
 <Element div at 0x1712c721088>,
 <Element div at 0x1712c721d88>,
 <Element div at 0x1712c721908>,
 <Element div at 0x1712c721288>,
 <Element div at 0x1712c721f88>,
 <Element div at 0x1712c721408>,
 <Element div at 0x1712c721fc8>,
 <Element div at 0x1712c6b66c8>,
 <Element div at 0x1712c6e8a88>,
 <Element div at 0x1712c6e86c8>,
 <Element div at 0x1712c6e85c8>,
 <Element div at 0x1712c6e8908>]

In [4]:
bs.xpath('//ol [@class = "grid_view"]/li[1]/div/div[2]/div[2]/div/span[2]')[0].text 

'9.6'

In [5]:
bs.xpath('//ol [@class = "grid_view"]/li[1]/div/div[2]/div[2]/p[2]/span')[0].text 

'希望让人自由。'

In [6]:
bs.xpath('//ol [@class = "grid_view"]/li[1]/div/div[2]/div[2]/div/span[4]')[0].text 

'1334437人评价'

In [27]:
import pandas as pd
import requests
from lxml import etree
import time


#循环构造网址
def format_url(base_url,pages = 10):
    urls = []
    for num in range(0,pages * 25,25):
        urls.append(base_url.format(num))
    return urls

#解析单个页面
def parse_page(url,headers):
    #创建一个存储结果的容器
    result = pd.DataFrame()
    html = requests.get(url,headers = headers)
    bs = etree.HTML(html.text)
    for i in bs.xpath('//div [@class = "item"]'):
        #电影中文名
        book_ch_name = i.xpath('div[2]/div[1]/a/span[1]')[0].text
        #评分
        score = i.xpath('div[2]/div[2]/div/span[2]')[0].text
        #电影信息
        book_info = i.xpath('div[2]/div[2]/p[1]/text()[1]')
        #评价数量由于数据不规整，这里用PYTHON字符串方法对数据进行了处理
        comment_num = i.xpath('div[2]/div[2]/div/span[4]')[0].text.replace(' ','').strip('(\n').strip('\n)')
        try:
            #防止没有一句话概括
            brief = i.xpath('div[2]/div[2]/p[2]/span')[0].text
        except:
            brief = None
        #这里的cache是存储每一次循环的结果，然后通过下一步操作循环更新result里面的数据
        cache = pd.DataFrame({'中文名':[book_ch_name],'评分':[score],\
                              '电影信息':[book_info],'评价数量':[comment_num],'一句话概括':[brief]})
        #把新循环中的cache添加到result下面
        result = pd.concat([result,cache])

    return result


def main():
    final_result = pd.DataFrame()
    base_url = 'https://movie.douban.com/top250?start={}'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    urls = format_url(base_url,pages = 10)
    for url in urls:
        res = parse_page(url,headers = headers)
        final_result = pd.concat([final_result,res])
        print(url)
    #爬虫要文明，这里设置了一个爱心520时间
    time.sleep(5.2)
    return final_result


if __name__ == "__main__":
    final_result = main()

https://movie.douban.com/top250?start=0
https://movie.douban.com/top250?start=25
https://movie.douban.com/top250?start=50
https://movie.douban.com/top250?start=75
https://movie.douban.com/top250?start=100
https://movie.douban.com/top250?start=125
https://movie.douban.com/top250?start=150
https://movie.douban.com/top250?start=175
https://movie.douban.com/top250?start=200
https://movie.douban.com/top250?start=225


In [32]:
final_result.reindex(columns=['中文名','电影信息','一句话概括','评价数量','评分'])

,中文名,电影信息,一句话概括,评价数量,评分
0,肖申克的救赎,[\n 导演: 弗兰克·德拉邦特 Fr...,希望让人自由。,1334437人评价,9.6
0,霸王别姬,[\n 导演: 陈凯歌 Kaige C...,风华绝代。,985345人评价,9.6
0,这个杀手不太冷,[\n 导演: 吕克·贝松 Luc B...,怪蜀黍和小萝莉不得不说的故事。,1220764人评价,9.4
0,阿甘正传,[\n 导演: 罗伯特·泽米吉斯 Ro...,一部美国近现代史。,1051154人评价,9.4
0,美丽人生,[\n 导演: 罗伯托·贝尼尼 Rob...,最美的谎言。,615039人评价,9.5
0,泰坦尼克号,[\n 导演: 詹姆斯·卡梅隆 Jam...,失去的才是永恒的。,989701人评价,9.3
0,千与千寻,[\n 导演: 宫崎骏 Hayao M...,最好的宫崎骏，最好的久石让。,978707人评价,9.3
0,辛德勒的名单,[\n 导演: 史蒂文·斯皮尔伯格 S...,拯救一个人，就是拯救整个世界。,548844人评价,9.5
0,盗梦空间,[\n 导演: 克里斯托弗·诺兰 Ch...,诺兰给了我们一场无法盗取的梦。,1063905人评价,9.3
0,忠犬八公的故事,[\n 导演: 莱塞·霍尔斯道姆 La...,永远都不能忘记你所爱的人。,695174人评价,9.3


In [33]:
final_result.to_csv('douban_moive_top250.csv')